In [13]:
import os
import mlflow
import psycopg2 as psycopg
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
import mlflow

# Предполагаем, что у нас уже есть датафрейм df с данными

# Подготовка данных
X = df.drop(['target', 'customer_id', 'begin_date', 'end_date'], axis=1)
y = df['target']

# Кодирование категориальных признаков
X = pd.get_dummies(X, drop_first=True)

# Разделение на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Создание и применение импутера
imputer = SimpleImputer(strategy='mean')
X_train_imputed = imputer.fit_transform(X_train)
X_test_imputed = imputer.transform(X_test)

# Масштабирование числовых признаков
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_imputed)
X_test_scaled = scaler.transform(X_test_imputed)

# Обучение модели
model = LogisticRegression(random_state=42)
model.fit(X_train_scaled, y_train)

# Получение предсказаний
prediction = model.predict(X_test_scaled)
proba = model.predict_proba(X_test_scaled)[:, 1]

# Определение глобальных переменных
EXPERIMENT_NAME = "churn_prediction_experiment"
RUN_NAME = "logistic_regression_baseline"

# Настройка MLflow
mlflow.set_experiment(EXPERIMENT_NAME)

print("Пререквизиты подготовлены:")
print("1. Объект модели сохранен в переменную 'model'")
print("2. Предсказанные значения сохранены в 'prediction' и 'proba'")
print("3. Истинные метки сохранены в 'y_test'")
print("4. Данные для предсказания сохранены в 'X_test_scaled'")
print("5. Глобальные переменные EXPERIMENT_NAME и RUN_NAME определены")


2025/08/03 18:02:09 INFO mlflow.tracking.fluent: Experiment with name 'churn_prediction_experiment' does not exist. Creating a new experiment.


Пререквизиты подготовлены:
1. Объект модели сохранен в переменную 'model'
2. Предсказанные значения сохранены в 'prediction' и 'proba'
3. Истинные метки сохранены в 'y_test'
4. Данные для предсказания сохранены в 'X_test_scaled'
5. Глобальные переменные EXPERIMENT_NAME и RUN_NAME определены


In [ ]:
# определяем основные credentials, которые нужны для подключения к MLflow
# важно, что credentials мы передаём для себя как пользователей Tracking Service
# у вас должен быть доступ к бакету, в который вы будете складывать артефакты
os.environ["MLFLOW_S3_ENDPOINT_URL"] = "https://storage.yandexcloud.net" #endpoint бакета от YandexCloud
os.environ["AWS_ACCESS_KEY_ID"] = os.getenv("AWS_ACCESS_KEY_ID") # получаем id ключа бакета, к которому подключён MLFlow, из .env
os.environ["AWS_SECRET_ACCESS_KEY"] = os.getenv("AWS_SECRET_ACCESS_KEY") # получаем ключ бакета, к которому подключён MLFlow, из .env


# определяем глобальные переменные
# поднимаем MLflow локально
TRACKING_SERVER_HOST = "127.0.0.1"
TRACKING_SERVER_PORT = 5000

YOUR_NAME = "ivan_panchenko" # введите своё имя для создания уникального эксперимента
assert YOUR_NAME, "введите своё имя в переменной YOUR_NAME для создания уникального эксперимента"

# название тестового эксперимента и запуска (run) внутри него
EXPERIMENT_NAME = f"log_model_{YOUR_NAME}"
RUN_NAME = "model_run"

# # тестовые данные
# METRIC_NAME = "test_metric"
# METRIC_VALUE = 0

# устанавливаем host, который будет отслеживать наши эксперименты
mlflow.set_tracking_uri(f"http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}")
mlflow.set_registry_uri(f"http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}") 



In [4]:
connection = {"sslmode": "require", "target_session_attrs": "read-write"}
postgres_credentials = {
    "host": os.getenv('DB_DESTINATION_HOST'),
    "port": os.getenv('DB_DESTINATION_PORT'), 
    "dbname": os.getenv('DB_DESTINATION_NAME'), 
    "user": os.getenv('DB_DESTINATION_USER'), 
    "password": os.getenv('DB_DESTINATION_PASSWORD')
}
assert all([var_value != "" for var_value in list(postgres_credentials.values())])

connection.update(postgres_credentials)

# определим название таблицы, в которой хранятся наши данные.
TABLE_NAME = "users_churn"

# эта конструкция создаёт контекстное управление для соединения с базой данных 
# оператор with гарантирует, что соединение будет корректно закрыто после выполнения всех операций 
# закрыто оно будет даже в случае ошибки, чтобы не допустить "утечку памяти"
with psycopg.connect(**connection) as conn:

# создаёт объект курсора для выполнения запросов к базе данных
# с помощью метода execute() выполняется SQL-запрос для выборки данных из таблицы TABLE_NAME
    with conn.cursor() as cur:
        cur.execute(f"SELECT * FROM {TABLE_NAME}")
                
                # извлекаем все строки, полученные в результате выполнения запроса
        data = cur.fetchall()

                # получает список имён столбцов из объекта курсора
        columns = [col[0] for col in cur.description]

# создаёт объект DataFrame из полученных данных и имён столбцов. 
# это позволяет удобно работать с данными в Python, используя библиотеку Pandas.
df = pd.DataFrame(data, columns=columns)

In [6]:
df.head()

,id,customer_id,begin_date,end_date,type,paperless_billing,payment_method,monthly_charges,total_charges,internet_service,...,device_protection,tech_support,streaming_tv,streaming_movies,gender,senior_citizen,partner,dependents,multiple_lines,target
0,17,8191-XWSZG,2015-10-01,NaT,One year,No,Mailed check,20.65,1022.95,None,...,None,None,None,None,Female,0,No,No,No,0
1,59,3957-SQXML,2017-04-01,NaT,Two year,No,Credit card (automatic),24.95,894.30,None,...,None,None,None,None,Female,0,Yes,Yes,Yes,0
2,148,6837-BJYDQ,2019-11-01,NaT,One year,No,Mailed check,19.60,61.35,None,...,None,None,None,None,Male,0,No,No,No,0
3,482,0486-LGCCH,2019-03-01,NaT,Two year,No,Mailed check,19.65,225.75,None,...,None,None,None,None,Male,0,Yes,Yes,No,0
4,1,7590-VHVEG,2020-01-01,NaT,Month-to-month,Yes,Electronic check,29.85,29.85,DSL,...,No,No,No,No,Female,0,Yes,No,None,0


In [20]:


# Предполагаем, что у нас уже есть датафрейм df с данными

# Подготовка данных
X = df.drop(['target', 'customer_id', 'begin_date', 'end_date'], axis=1)
y = df['target']

# Кодирование категориальных признаков
X = pd.get_dummies(X, drop_first=True)

# Разделение на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Создание и применение импутера
imputer = SimpleImputer(strategy='mean')
X_train_imputed = imputer.fit_transform(X_train)
X_test_imputed = imputer.transform(X_test)

# Масштабирование числовых признаков
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_imputed)
X_test_scaled = scaler.transform(X_test_imputed)

# Обучение модели
model = LogisticRegression(random_state=42)
model.fit(X_train_scaled, y_train)

# Получение предсказаний
prediction = model.predict(X_test_scaled)
probas = model.predict_proba(X_test_scaled)[:, 1]

# Определение глобальных переменных
EXPERIMENT_NAME = "churn_prediction_experiment"
RUN_NAME = "logistic_regression_baseline"

# Настройка MLflow
mlflow.set_experiment(EXPERIMENT_NAME)

print("Пререквизиты подготовлены:")
print("1. Объект модели сохранен в переменную 'model'")
print("2. Предсказанные значения сохранены в 'prediction' и 'proba'")
print("3. Истинные метки сохранены в 'y_test'")
print("4. Данные для предсказания сохранены в 'X_test_scaled'")
print("5. Глобальные переменные EXPERIMENT_NAME и RUN_NAME определены")

Пререквизиты подготовлены:
1. Объект модели сохранен в переменную 'model'
2. Предсказанные значения сохранены в 'prediction' и 'proba'
3. Истинные метки сохранены в 'y_test'
4. Данные для предсказания сохранены в 'X_test_scaled'
5. Глобальные переменные EXPERIMENT_NAME и RUN_NAME определены


In [21]:
proba

array([0.06878521, 0.29417414, 0.01586858, ..., 0.2023295 , 0.42400973,
       0.04200614])

In [22]:
from sklearn.metrics import (
    roc_auc_score, f1_score, precision_score, recall_score,
    confusion_matrix, log_loss
)

# заведите словарь со всеми метриками
metrics = {}

# посчитайте метрики из модуля sklearn.metrics
# err_1 — ошибка первого рода
# err_2 — ошибка второго рода
_, err1, _, err2 = confusion_matrix(y_test, prediction, normalize='all').ravel()

cm = confusion_matrix(y_test, prediction)
tn, fp, fn, tp = cm.ravel()
err1 = fp / (fp + tn)  # False Positive Rate
err2 = fn / (fn + tp)  # False Negative Rate
auc = roc_auc_score(y_test, probas)
precision = precision_score(y_test, prediction)
recall = recall_score(y_test, prediction)
f1 = f1_score(y_test, prediction)
logloss = log_loss(y_test, prediction)

# запишите значения метрик в словарь
metrics["err1"] = err1
metrics["err2"] = err2
metrics["auc"] = auc
metrics["precision"] = precision
metrics["recall"] = recall
metrics["f1"] = f1
metrics["logloss"] = logloss

In [23]:
metrics

{'err1': 0.09803921568627451,
 'err2': 0.4910025706940874,
 'auc': 0.8339482836836534,
 'precision': 0.6644295302013423,
 'recall': 0.5089974293059126,
 'f1': 0.5764192139737991,
 'logloss': 7.444076037071037}